In [1]:
## load rtweet and tidyverse
library(rtweet)
suppressPackageStartupMessages(library(tidyverse))

Warning message:
"package 'rtweet' was built under R version 3.4.3"Warning message:
"package 'tidyverse' was built under R version 3.4.3"Warning message:
"package 'tibble' was built under R version 3.4.3"Warning message:
"package 'tidyr' was built under R version 3.4.3"Warning message:
"package 'purrr' was built under R version 3.4.3"Warning message:
"package 'dplyr' was built under R version 3.4.2"

In [2]:
## search for users using provided queries

breitbart <- search_users("breitbart", n = 1, verbose = FALSE)
thinkprogress <- search_users("thinkprogress", n = 1, verbose = FALSE)
reuters  <- search_users("reuterspolitics", n = 1, verbose = FALSE)

In [3]:
head(breitbart)

code,message
32,Could not authenticate you.


In [4]:
head(thinkprogress)

code,message
32,Could not authenticate you.


In [5]:
head(reuters)

code,message
32,Could not authenticate you.


In [6]:
## complete the code below
gop <- get_followers("BreitbartNews", n = 5000)
head(gop, 2)

Warning message:
"32 - Could not authenticate you."

ERROR: Error in `$<-.data.frame`(`*tmp*`, app, value = "rtweet_rstats"): replacement has 1 row, data has 0


In [7]:
## complete the code below
dem <- get_followers("thinkprogress", n = 5000)
head(dem, 2)

Warning message:
"32 - Could not authenticate you."

ERROR: Error in `$<-.data.frame`(`*tmp*`, "app", value = "rtweet_rstats"): replacement has 1 row, data has 0


In [8]:
## complete the code below
mod <- get_followers("ReutersPolitics", n = 5000)
head(mod, 2)

Warning message:
"32 - Could not authenticate you."

ERROR: Error in `$<-.data.frame`(`*tmp*`, "app", value = "rtweet_rstats"): replacement has 1 row, data has 0


In [9]:
## complete the code below

## get gop users data
gop_data <- lookup_users(gop$user_id)

ERROR: Error in lookup_users(gop$user_id): object 'gop' not found


In [10]:
## complete the code below

## get dem users data
dem_data <- lookup_users(dem$user_id)

ERROR: Error in lookup_users(dem$user_id): object 'dem' not found


In [11]:
## complete the code below

## get mod users data
mod_data <- lookup_users(mod$user_id)

ERROR: Error in lookup_users(mod$user_id): object 'mod' not found


In [12]:
## complete the code below

## create "partisan" grouping variable
gop_data$partisan <- "gop"
dem_data$partisan <- "dem"
mod_data$partisan <- "mod"

## merge into single data frame
data <- bind_rows(gop_data, dem_data, mod_data)

ERROR: Error in gop_data$partisan <- "gop": object 'gop_data' not found


In [13]:
## complete the code below

## PRINT: group by partisan and summarise integer vars
data %>%
    group_by(partisan) %>%
    summarize(friends = mean(friends_count, na.rm = TRUE),
               followers = mean(followers_count, na.rm = TRUE))

ERROR: Error in UseMethod("group_by_"): no applicable method for 'group_by_' applied to an object of class "function"


In [14]:
## complete the code below

## PLOT: group by partisan and summarise integer vars

data %>%
    mutate(Following = factor(partisan, labels = c("thinkprogress", "BreitbartNews", "ReutersPolitics"))) %>%
    group_by(Following) %>%
    summarize(
        friends = mean(friends_count, na.rm = TRUE),
        followers = mean(followers_count, na.rm = TRUE)) %>%
    gather(var, count, -Following) %>%
    ggplot(aes(x = Following, y = count, fill = Following)) + 
    geom_col() + 
    facet_wrap(~ var, ncol = 2, scales = "free_y") +
    theme_bw(base_size = 10) + 
    theme(plot.title = element_text(face = "bold")) + 
    scale_fill_manual(values = c(BreitbartNews = "red2", thinkprogress = "blue", ReutersPolitics = "purple")) + 
    labs(
            title = "User stats for followers of thinkprogress, ReutersPolitics, and BreitbartNews",
            subtitle = "Mean number of friends and followers by group (n = 1000)",
            x = NULL, y = "Count"
    )

ERROR: Error in UseMethod("mutate_"): no applicable method for 'mutate_' applied to an object of class "function"


In [15]:
## complete the code below


rsamp_uq <- function(..., n) {
    
    dots <- list(...)
    
    groups <- names(dots)
    
    user_ids <- lapply(dots, "[[", "user_id")
    
    
    for (i in seq_along(dots)) {
        
        ## filter out users found in other groups and protected users
        dots[[i]] <- filter(
            dots[[i]], 
            !user_id %in% unlist(user_ids[-i]) & !protected &verified
        )
        
        dots[[i]]$group <- groups[i]
    }
    
    sample_users <- function(x, n) x[x$user_id %in% sample(x$user_id, n), ]
        
    out <- lapply(dots, sample_users, n)
        
    bind_rows(out)
}
    
    
    
smps <- rsamp_uq(gop = gop_data, dem = dem_data, mod = mod_data, n = 5)

ERROR: Error in rsamp_uq(gop = gop_data, dem = dem_data, mod = mod_data, n = 5): object 'gop_data' not found


In [16]:
head(smps, 15)

ERROR: Error in head(smps, 15): object 'smps' not found


In [17]:
nrow(smps)

ERROR: Error in nrow(smps): object 'smps' not found


In [18]:
followers <- lapply(smps$user_id, get_followers, verbose = FALSE)

ERROR: Error in lapply(smps$user_id, get_followers, verbose = FALSE): object 'smps' not found


In [19]:
smps$followers <- I(followers)
head(smps, 3)

ERROR: Error in structure(x, class = unique(c("AsIs", oldClass(x)))): object 'followers' not found


In [20]:
get_top_3 <- function (x, n = 3){
    tbl <- sort(table(unlist(x)), decreasing = TRUE)
    
    if (length(tbl) > n) {
        tbl <- tbl[seq_len(n)]
    }    
        
    sns <- lookup_users(names(tbl))
    sns$screen_name
}

In [21]:
smps %>% 
    filter(length(smps$followers) >= 3) %>% 
    group_by(group) %>%
    summarise(
        freq = list(get_top_3(screen_name))
    )

ERROR: Error in eval(lhs, parent, parent): object 'smps' not found
